In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Agregamos la libreria de pyAgrum.

In [ ]:
!pip install pyAgrum pydotplus


# Añadimos librerias varias 

In [ ]:
import pandas
import os
import math
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
from pyAgrum.lib.bn2roc import showROC_PR
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import pandas as pd
import numpy as np 

from os import listdir
from os.path import isfile, join

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model

import warnings  
warnings.filterwarnings('ignore')

# Train y Test

In [ ]:
train=pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test=pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')

# Informacion del Dataset

# ATRIBUTOS DEL DATASET

* image_name - unique identifier, points to filename of related DICOM image
* patient_id - unique patient identifier
* sex - the sex of the patient (when unknown, will be blank)
* age_approx - approximate patient age at time of imaging
* anatom_site_general_challenge - location of imaged site
* diagnosis - detailed diagnosis information (train only)
* benign_malignant - indicator of malignancy of imaged lesion
* target - binarized version of the target variable

In [ ]:
train.head()

In [ ]:
train.info()

# Seguimos realizando un preratamiento, ahora filtramos las variables con diversa cantidad de valores, al ser las redes bayesianas redes discretas filtramos cualquier variable con mas de 15resultados diferentes.

In [ ]:
for k in train.keys():
    print('{0}: {1}'.format(k, len(train[k].unique())))
   

In [ ]:
for k in train.keys():
    if len(train[k].unique())<=15:
        print(k)

# Categorizamos la edad para poder tratarla.

In [ ]:
def for_age(row):
    try:
        age = float(row['age_approx'])
        if age < 18:
            return 'teen'
        elif age < 30:
            return 'twenties'
        elif age < 40:
            return 'thirties'
        elif age < 50:
            return 'forties'
        elif age < 60:
            return 'fifties'
        elif age < 70:
            return 'sixties'
        else:
            return 'old'
    except ValueError:
        return np.nan
    
def for_boolean(row, col):
    try:
        val = int(row[col])
        if row[col] >= 1:
            return "True"
        else:
            return "False"
    except ValueError:
        return "False"

In [ ]:
def pretreat(df):
    if 'target' in df.columns:
        df['target'] = df.apply(lambda row: for_boolean(row, 'target'), axis=1)
    df['age_approx'] = df.apply(for_age, axis=1)
    return df

In [ ]:
train_df = pretreat(train)
test_df=pretreat(test)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

Bayes Theorem
P(A|B)=P(B|A)P(A)P(B)(1)
The above formula can be reinterpreted in the machine learning terms of features and class label. Classification is a problem of assigning classes based on the features provided.

P(class|features)=P(features|class)P(class)P(features)(2)
For example we need to classify a person's sex based on the height and weight. So here the class={male,female} and features={height,weight}, and the formula can he re-written as :

P(sex|height,weight)=P(height,weight|sex)P(sex)P(height,weight)(3)
Based on these information lets go ahead and implement the algorithm on a discretized problem using the vectorization properties of the problem. To understand what is vectorization of problem please read From Python to Numpy.
REFERENCIA: http://deebuls.github.io/Naive-Bayes-Pandas.html

Calculamos primero los apriori 

Calculating the prior
prior = P(maligno/benigno)

In [ ]:
prior = train.groupby('benign_malignant').size().div(len(train)) #count()['Age']/len(data)
prior

Ahora calculamos la probabilidad(likelihood).

Likelihood is generated for each of the features of the dataset. Basicallay likelihood is probability of finding each feature given class label.

P(age_approx|benign_malignant)(6)
P(sex|benign_malignant)(7)
P(anatom_site_general_challenge|benign_malignant)(8)
P(diagnosis|benign_malignant)(9)

In [ ]:
likelihood = {}
likelihood['age_approx'] = train.groupby(['benign_malignant', 'age_approx']).size().div(len(train)).div(prior)
likelihood['sex'] = train.groupby(['benign_malignant', 'sex']).size().div(len(train)).div(prior)
likelihood['anatom_site_general_challenge'] = train.groupby(['benign_malignant', 'anatom_site_general_challenge']).size().div(len(train)).div(prior)

likelihood

Ahora calculamos el posteriori.Necesitamos saber si una persona tendra cancer benigno o maligno en base a la nueva informacion



In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER MALIGNO EN EL TORSO SEA HOMBRE 
#p_maligno = likelihood['age_approx']['malignant']['twenties'] * likelihood['sex']['malignant']['male'] * \likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \ * prior['malignant']
p_maligno_m_torso = likelihood['age_approx']['malignant'] * likelihood['sex']['malignant']['male'] * \
        likelihood['anatom_site_general_challenge']['malignant']['torso'] \
        * prior['malignant']

print('Posteriori 1 : Torso maligno hombre',p_maligno_m_torso)



In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER BENIGNO EN EL TORSO SEA HOMBRE 
p_benigno_m_torso = likelihood['age_approx']['benign'] * likelihood['sex']['benign']['male'] * \
        likelihood['anatom_site_general_challenge']['benign']['torso'] \
        * prior['benign']

print('Posteriori 2: Torso benigno hombre',p_benigno_m_torso)


In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER MALIGNO EN EL TORSO SEA MUJER 
p_maligno_f_torso = likelihood['age_approx']['malignant'] * likelihood['sex']['malignant']['female'] * \
        likelihood['anatom_site_general_challenge']['malignant']['torso'] \
        * prior['malignant']

print('Posteriori 3 Torso mujer maligno',p_maligno_f_torso)


In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER BENIGNO EN EL TORSO SEA MUJER
p_benigno_f_torso = likelihood['age_approx']['benign'] * likelihood['sex']['benign']['female'] * \
        likelihood['anatom_site_general_challenge']['benign']['torso'] \
        * prior['benign']

print('Posteriori 4 Torso mujer benigno',p_benigno_f_torso)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER MALIGNO EN LA CABEZA SEA HOMBRE 
#p_maligno = likelihood['age_approx']['malignant']['twenties'] * likelihood['sex']['malignant']['male'] * \likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \ * prior['malignant']
p_maligno_m_head = likelihood['age_approx']['malignant'] * likelihood['sex']['malignant']['male'] * \
        likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \
        * prior['malignant']

print('Posteriori 5: Cabeza maligno hombre',p_maligno_m_head)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER BENIGNO EN EL TORSO SEA HOMBRE 
p_benigno_m_head = likelihood['age_approx']['benign'] * likelihood['sex']['benign']['male'] * \
        likelihood['anatom_site_general_challenge']['benign']['head/neck'] \
        * prior['benign']

print('Posteriori 6 cabeza benigno hombre',p_benigno_m_head)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER MALIGNO EN LA CABEZA SEA MUJER 
#p_maligno = likelihood['age_approx']['malignant']['twenties'] * likelihood['sex']['malignant']['male'] * \likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \ * prior['malignant']
p_maligno_f_head = likelihood['age_approx']['malignant'] * likelihood['sex']['malignant']['female'] * \
        likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \
        * prior['malignant']

print('Posteriori 7 cabeza maligno mujer',p_maligno_f_head)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER BENIGNO EN LA CABEZA SEA MUJER 
p_benigno_f_head = likelihood['age_approx']['benign'] * likelihood['sex']['benign']['female'] * \
        likelihood['anatom_site_general_challenge']['benign']['head/neck'] \
        * prior['benign']

print('Posteriori 8 cabeza benigno mujer',p_benigno_f_head)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER MALIGNO EN LAS EXTREMIDADES INFERIORES SEA HOMBRE 
#p_maligno = likelihood['age_approx']['malignant']['twenties'] * likelihood['sex']['malignant']['male'] * \likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \ * prior['malignant']
p_maligno_m_l_e = likelihood['age_approx']['malignant'] * likelihood['sex']['malignant']['male'] * \
        likelihood['anatom_site_general_challenge']['malignant']['lower extremity'] \
        * prior['malignant']

print('Posteriori 9 extremidades inferiores maligno hombre',p_maligno_m_l_e)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER BENIGNO EN LAS EXTREMIDADES INFERIORES SEA HOMBRE 
p_benigno_m_l_e = likelihood['age_approx']['benign'] * likelihood['sex']['benign']['male'] * \
        likelihood['anatom_site_general_challenge']['benign']['lower extremity'] \
        * prior['benign']

print('Posteriori 10 extremidades inferiores benigno hombre',p_benigno_m_l_e)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER MALIGNO EN LAS EXTREMIDADES INFERIORES SEA MUJER 
#p_maligno = likelihood['age_approx']['malignant']['twenties'] * likelihood['sex']['malignant']['male'] * \likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \ * prior['malignant']
p_maligno_f_l_e = likelihood['age_approx']['malignant'] * likelihood['sex']['malignant']['female'] * \
        likelihood['anatom_site_general_challenge']['malignant']['lower extremity'] \
        * prior['malignant']

print('Posteriori 11, maligno mujer extremidades inferiores',p_maligno_f_l_e)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER BENIGNO EN LAS EXTREMIDADES INFERIORES SEA MUJER 
p_benigno_f_l_e = likelihood['age_approx']['benign'] * likelihood['sex']['benign']['female'] * \
        likelihood['anatom_site_general_challenge']['benign']['lower extremity'] \
        * prior['benign']

print('Posteriori 12 benigno mujer extremidades inferiores',p_benigno_f_l_e)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER MALIGNO EN LAS EXTREMIDADES INFERIORES SEA HOMBRE 
#p_maligno = likelihood['age_approx']['malignant']['twenties'] * likelihood['sex']['malignant']['male'] * \likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \ * prior['malignant']
p_maligno_m_oral = likelihood['age_approx']['malignant'] * likelihood['sex']['malignant']['male'] * \
        likelihood['anatom_site_general_challenge']['malignant']['oral/genital'] \
        * prior['malignant']

print('Posteriori 13 maligno hombre en boca',p_maligno_m_oral)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER BENIGNO EN LA BOCA INFERIORES SEA HOMBRE 
p_benigno_m_oral = likelihood['age_approx']['benign'] * likelihood['sex']['benign']['male'] * \
        likelihood['anatom_site_general_challenge']['benign']['oral/genital'] \
        * prior['benign']

print('Posteriori 14 benigno hombre en boca',p_benigno_m_oral)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER MALIGNO EN LAS BOCA  SEA MUJER 
#p_maligno = likelihood['age_approx']['malignant']['twenties'] * likelihood['sex']['malignant']['male'] * \likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \ * prior['malignant']
p_maligno_f_oral = likelihood['age_approx']['malignant'] * likelihood['sex']['malignant']['female'] * \
        likelihood['anatom_site_general_challenge']['malignant']['lower extremity'] \
        * prior['malignant']

print('Posteriori 15, maligno mujer oral',p_maligno_f_oral)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER BENIGNO EN LA BOCA SEA MUJER 
p_benigno_f_oral = likelihood['age_approx']['benign'] * likelihood['sex']['benign']['female'] * \
        likelihood['anatom_site_general_challenge']['benign']['oral/genital'] \
        * prior['benign']

print('Posteriori 16 benigno mujer, oral',p_benigno_f_oral)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER MALIGNO EN LAS EXTREMIDADES INFERIORES SEA HOMBRE 
#p_maligno = likelihood['age_approx']['malignant']['twenties'] * likelihood['sex']['malignant']['male'] * \likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \ * prior['malignant']
p_maligno_m_u_e = likelihood['age_approx']['malignant'] * likelihood['sex']['malignant']['male'] * \
        likelihood['anatom_site_general_challenge']['malignant']['upper extremity'] \
        * prior['malignant']

print('Posteriori 17 maligno hombre extremidades superiores',p_maligno_m_u_e)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER BENIGNO EN LA BOCA INFERIORES SEA HOMBRE 
p_benigno_m_u_e = likelihood['age_approx']['benign'] * likelihood['sex']['benign']['male'] * \
        likelihood['anatom_site_general_challenge']['benign']['upper extremity'] \
        * prior['benign']

print('Posteriori 18, benigno hombre, extremidades superiores',p_benigno_m_u_e)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER MALIGNO EN LAS BOCA  SEA MUJER 
#p_maligno = likelihood['age_approx']['malignant']['twenties'] * likelihood['sex']['malignant']['male'] * \likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \ * prior['malignant']
p_maligno_f_u_e = likelihood['age_approx']['malignant'] * likelihood['sex']['malignant']['female'] * \
        likelihood['anatom_site_general_challenge']['malignant']['upper extremity'] \
        * prior['malignant']

print('Posteriori 19 maligno mujer extremidades superiores',p_maligno_f_u_e)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER BENIGNO EN LAS EXTREMIDADES SUPERIORES SEA MUJER 
p_benigno_f_u_e = likelihood['age_approx']['benign'] * likelihood['sex']['benign']['female'] * \
        likelihood['anatom_site_general_challenge']['benign']['upper extremity'] \
        * prior['benign']

print('Posteriori 20 benigno mujer extremidades superiores',p_benigno_f_u_e)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER MALIGNO EN LAS MANOS SEA HOMBRE 
#p_maligno = likelihood['age_approx']['malignant']['twenties'] * likelihood['sex']['malignant']['male'] * \likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \ * prior['malignant']
p_maligno_m_manos = likelihood['age_approx']['malignant'] * likelihood['sex']['malignant']['male'] * \
        likelihood['anatom_site_general_challenge']['malignant']['palms/soles'] \
        * prior['malignant']

print('Posteriori 21 maligno hombre manos',p_maligno_m_manos)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER BENIGNO EN LA MANOS INFERIORES SEA HOMBRE 
p_benigno_m_manos = likelihood['age_approx']['benign'] * likelihood['sex']['benign']['male'] * \
        likelihood['anatom_site_general_challenge']['benign']['palms/soles'] \
        * prior['benign']

print('Posteriori 22 benigno hombre manos' ,p_benigno_m_manos)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER MALIGNO EN LAS MANOS  SEA MUJER 
#p_maligno = likelihood['age_approx']['malignant']['twenties'] * likelihood['sex']['malignant']['male'] * \likelihood['anatom_site_general_challenge']['malignant']['head/neck'] \ * prior['malignant']
p_maligno_f_manos = likelihood['age_approx']['malignant'] * likelihood['sex']['malignant']['female'] * \
        likelihood['anatom_site_general_challenge']['malignant']['palms/soles'] \
        * prior['malignant']

print('Posteriori 23 maligno mujer manos',p_maligno_f_manos)

In [ ]:
#PROBABILIDAD DE QUE UNA PERSONA TENGA CANCER BENIGNO EN LAS MANOS SEA MUJER 
p_benigno_f_manos = likelihood['age_approx']['benign'] * likelihood['sex']['benign']['female'] * \
        likelihood['anatom_site_general_challenge']['benign']['palms/soles'] \
        * prior['benign']

print('Posteriori 24 benigno mujer manos',p_benigno_f_manos)

# Creamos la red bayesiana

In [ ]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
%matplotlib inline 


bn = gum.BayesNet("Melanoma")
bn =gum.fastBN("target{False|True}->age{teen|twenties|thirties|forties|fifties|sixties|old}->location{head/neck|lower extremity|oral/genital|palms/soles|torso|upper extremity}->sex{female|male}")
print(bn.variable("target"))
print(bn.variable("age"))
print(bn.variable("sex"))
print(bn.variable("location"))

bn

# Prevalencia de benigno frente a maligno..

In [ ]:
train_df.groupby(['target']).size()/len(train_df)

Podemos decir que la posibilad de tener un cancer maligno es de 98:2, con lo que partimos de esa hipotemos.

In [ ]:
bn.cpt('target')[:] = [0.98237, 0.01763]
bn.cpt('target').normalizeAsCPT()
bn.cpt('target')

Obtenemos la prevalencia y agrumanos segun edades.

In [ ]:
train_df.groupby(['age_approx', 'target']).size()/len(train_df)

In [ ]:
bn.cpt('age')[{'target':0}] = [0.004498, 0.065719, 0.155739,0.240415,  0.240808, 0.170018,0.105174]
bn.cpt('age')[{'target':1}] = [0.000060,0.000664, 0.001479, 0.002355, 0.003532,0.004075, 0.005464]
bn.cpt('age')

Obtenemos la prevalencia en la relacion entre el sexo y la localizacion.


In [ ]:
train_df.groupby(['sex', 'anatom_site_general_challenge']).size()/len(train_df)

# location vs target

In [ ]:
bn.cpt('sex').fillWith = [[0.025177, 0.030580], [0.134305, 0.119242], [0.001358,  0.002385],
                          [0.004558,  0.006762],[0.228823, 0.279116], [0.081115,  0.068707],]
bn.cpt('sex')